In [1]:
import os
from pathlib import Path
import sys
import os
import _pickle as pickle
import json
import copy 
import pandas
import numpy as np
import matplotlib.pyplot as plt 
from matplotlib.backends.backend_pdf import PdfPages
import csv
import json
from collections import Counter
import statistics 
import plotly.graph_objects as go
import math

In [2]:
with open('../../responseJudgement.json') as json_file:
    judgements = json.load(json_file)
    
with open('../../responseJudgement_sugg.json') as json_file:
    suggJudgements = json.load(json_file)

In [3]:
len(judgements)

15417

In [4]:
len(suggJudgements)

1014

In [5]:
cleanPath = '../../clean/'
settingName = 'Setting 5'

In [6]:
responseSet = {}

for root, dirs, files in os.walk(cleanPath):
    for file in files:
        if file.endswith(".json"):
            fileName = os.path.join(root, file)
            with open(fileName) as json_file:
                try:
                    data = json.load(json_file)
                    currFileName = data['currentFileName']
                    dialogs = data['dialogs']
                    
                    for i in range(1, len(dialogs)):
                        context = dialogs[i]['context']
                        uniqueStr = 'fileIndex=' + str(context['fileIndex']) + '&&contextLen=' + str(context['contextLen']) + '&&userResponse=' + context['userResponse'].strip()
                        
                        if(uniqueStr in responseSet):
                            continue
                        
                        else:
                            
                            foundMatches = [x for x in judgements if x['Input.contextLen'] == str(context['contextLen']) and x['Input.fileIndex'] == str(context['fileIndex'])]
                            foundJudgement = [x for x in foundMatches if x['Input.fileName'] == currFileName and x['Input.localIndex'] == str(context['localIndex'])]
                            
                            if(len(foundJudgement) > 0):
                                responseSet[uniqueStr] = foundJudgement
                        
                except Exception as e: print(e)

In [7]:
len(responseSet)

5139

In [8]:
suggSet = {}

for root, dirs, files in os.walk(cleanPath):
    for file in files:
        if file.endswith(".json"):
            fileName = os.path.join(root, file)
            with open(fileName) as json_file:
                try:
                    data = json.load(json_file)
                    currFileName = data['currentFileName']
                    dialogs = data['dialogs']
                    
                    for i in range(1, len(dialogs)):
                        context = dialogs[i]['context']
                        suggestions = context['suggestions']
                        
                        for sugg in suggestions:
                            uniqueStr = 'fileIndex=' + str(context['fileIndex']) + '&&contextLen=' + str(context['contextLen']) + '&&userResponse=' + sugg.strip()

                            if(uniqueStr in responseSet or uniqueStr in suggSet):
                                continue

                            else:

                                foundMatches = [x for x in suggJudgements if x['Input.contextLen'] == str(context['contextLen']) and x['Input.fileIndex'] == str(context['fileIndex'])]
                                foundJudgement = [x for x in foundMatches if x['Input.fileName'] == currFileName and x['Input.localIndex'] == str(context['localIndex'])]

                                if(len(foundJudgement) > 0):
                                    suggSet[uniqueStr] = foundJudgement
                        
                except Exception as e: print(e)

In [9]:
len(suggSet)

458

In [10]:
Qualtype = 'Correctness'
metric = {
    'Answer.' + Qualtype +'.Strongly Disagree': 1,
    'Answer.' + Qualtype +'.Disagree': 2,
    'Answer.' + Qualtype +'.Neutral': 3,
    'Answer.' + Qualtype +'.Agree': 4,
    'Answer.' + Qualtype +'.Strongly Agree': 5
}

In [11]:
scoresHuman = []

for root, dirs, files in os.walk(cleanPath + 'Setting 1'):
    for file in files:
        if file.endswith(".json"):
            fileName = os.path.join(root, file)
            with open(fileName) as json_file:
                try:
                    data = json.load(json_file)
                    dialogs = data['dialogs']
                    
                    for i in range(1, len(dialogs)):
                        context = dialogs[i]['context']
                        
                        userResponse = context['userResponse']
                        fileIndex = context['fileIndex']
                        contextLen = context['contextLen']
                        
                        identifierStr = 'fileIndex=' + str(fileIndex) + '&&contextLen=' + str(contextLen) + '&&userResponse=' + userResponse.strip()
                        
                        responseJudgements =  responseSet[identifierStr]
                        
                        currFeedback = []
                        for feedback in responseJudgements:
                            for key in feedback:
                                
                                if('Answer.' + Qualtype in key and feedback[key] == 'true'):
                                    currFeedback.append(metric[key])
                        
                        scoresHuman.append(statistics.mean(currFeedback))
                            
                        
                    
                except Exception as e: print(e)                 

In [12]:
scoresBot = []

for root, dirs, files in os.walk(cleanPath + settingName):
    for file in files:
        if file.endswith(".json"):
            fileName = os.path.join(root, file)
            with open(fileName) as json_file:
                try:
                    data = json.load(json_file)
                    dialogs = data['dialogs']
                    
                    for i in range(1, len(dialogs)):
                        context = dialogs[i]['context']
                        
                        userResponse = context['userResponse']
                        fileIndex = context['fileIndex']
                        contextLen = context['contextLen']
                        suggestions = context['suggestions']
                        
                        identifierStr = 'fileIndex=' + str(fileIndex) + '&&contextLen=' + str(contextLen) + '&&userResponse=' + suggestions[0].strip()
                        
                        if(identifierStr in responseSet):
                            responseJudgements =  responseSet[identifierStr]
                        else:
                            responseJudgements =  suggSet[identifierStr]
                        
                        currFeedback = []
                        for feedback in responseJudgements:
                            for key in feedback:
                                
                                if('Answer.' + Qualtype in key and feedback[key] == 'true'):
                                    currFeedback.append(metric[key])
                        
                        scoresBot.append(statistics.mean(currFeedback))
                            
                        
                    
                except Exception as e: print(e)                 

In [13]:
sugg1 = []
sugg3 = []
sugg5 = []

autoSugg1 = []
autoSugg3 = []
autoSugg5 = []


for root, dirs, files in os.walk(cleanPath + settingName):
    for file in files:
        if file.endswith(".json"):
            fileName = os.path.join(root, file)
            with open(fileName) as json_file:
                try:
                    data = json.load(json_file)
                    dialogs = data['dialogs']
                    
                    for i in range(1, len(dialogs)):
                        context = dialogs[i]['context']
                        
                        userResponse = context['userResponse']
                        fileIndex = context['fileIndex']
                        contextLen = context['contextLen']
                        
                        identifierStr = 'fileIndex=' + str(fileIndex) + '&&contextLen=' + str(contextLen) + '&&userResponse=' + userResponse.strip()
                        
                        responseJudgements =  responseSet[identifierStr]
                        
                        currFeedback = []
                        
                        for feedback in responseJudgements:
                            for key in feedback:
                                
                                if('Answer.' + Qualtype in key and feedback[key] == 'true'):
                                    currFeedback.append(metric[key])
                        
                        
                        if(data['autoFilledSuggestion'] == 0):
                            
                            if(data['suggestionsNumber'] == 1):
                                sugg1.append(statistics.mean(currFeedback))

                            elif(data['suggestionsNumber'] == 3):
                                sugg3.append(statistics.mean(currFeedback))

                            elif(data['suggestionsNumber'] == 5):
                                sugg5.append(statistics.mean(currFeedback))
                        
                        else:

                            if(data['suggestionsNumber'] == 1):
                                autoSugg1.append(statistics.mean(currFeedback))

                            elif(data['suggestionsNumber'] == 3):
                                autoSugg3.append(statistics.mean(currFeedback))
                            
                            elif(data['suggestionsNumber'] == 5):
                                autoSugg5.append(statistics.mean(currFeedback))
                    
                    
                except Exception as e: print(e)

In [19]:
colors = ['#B0C4DE', '#A9A9A9', '#AFEEEE', '#8FBC8F']

x = ['Human Only', 'Chatbot Only', 'C1', 'C2']
y = [scoresHuman, scoresBot, sugg1, autoSugg1]

yErorr = [np.std(z)/math.sqrt(len(z)) for z in y] 

y = [round(statistics.mean(z), 2) for z in y]

fig = go.Figure()
fig.add_trace(go.Bar(
    name='Setting 5',
    x=x, y=y, text=y, textposition='inside',insidetextanchor='middle',
    marker_color=colors,
    marker_line_color=colors, marker_line_width=1.5, opacity=0.8,
    error_y=dict(type='data', array=yErorr)
))

fig.update_layout(
    title={
        'text': 'Response correctness in Setting 5',
        'y':0.85,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

fig.show()